In [ ]:
import sys
sys.path.append('../src') 

# Importing libraries
import pandas as pd
import numpy as np

# Libraries for machine learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Libraries for plotting curves
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from itertools import cycle

# Importing script
import etl as etl
import visualize_data as visualize_data

import warnings
warnings.filterwarnings('ignore')

## Disease Risk Classification Model

The purpose of this notebook is to explore a variety of models and tune them in order to figure out which is best for disease risk classification. According to the article "Comparing different supervised machine learning algorithms for disease prediction" by Shahadat Uddin et al. (https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-019-1004-8), Support Vector Machines, Naive Bayes, and Random Forest are some of the most common machine learning algorithms applied to disease prediction. We will explore these along with other algorithms such as Logistic Regression and K Nearest Neighbors in this notebook.

Let's get straight into it and simulate a data set of 5,000 individuals.

In [ ]:
simulated_gwas_fp = '../testdata/gwas/gwas_simulate.tsv' 
maf_fp = '../references/snp_mafs.txt'
simulated_data = etl.simulate_data(simulated_gwas_fp, maf_fp, 5000)
simulated_data.head()

We will not be using the simulated data above for building the model. Given that the class label (disease risk category) was assigned to an individual depending on the weighted sum of all and only the SNPs in this data set, it would be too easy for a machine learning model to figure this out. Essentially, the above data set is a simulated 'ground truth'. In machine learning problems you typically don't work with all the variables that determine the label so something needs to be done about this.

As a solution, we will be making classifications based on a subset of SNPs in the above data set. To do this, we will be using a separate GWAS to inform us of SNPs that are most important in predicting disease. Only those SNPs that are in the above data set and the new GWAS will be used. The model will then be trained on this subset.

Let's load in the other GWAS data and filter the above data.

In [ ]:
model_gwas_fp = '../testdata/gwas/gwas_model.tsv' 
model_data = pd.read_csv(model_gwas_fp, sep='\t')
subset = set(simulated_data.columns).intersection(model_data['SNPS'].unique())
new_columns = list(subset)+['Class']

data = simulated_data[new_columns]

Now let's create a training and test set on the above data.

In [ ]:
X = data.drop('Class', axis=1)
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [ ]:
# get proportion of each class
prop_per_class = y.value_counts(normalize=True)
prop_per_class

Now we can train different models on these sets and assess their results.

**Note:**

>It is important to note that accuracy is not the only metric that is important here to assess the performance of the models. Since we are working with predicting the disease risk of individuals, **Type I** and **Type II** errors are also very important. It is potentially dangerous to classify an individual as low risk when they are actually high risk (Type II, False Negative). Additionally, classifying an individual as high risk when they are actually low risk could cause the individual some unecessary stress within themselves and their families (Type I, False Positive). Therefore, it is important to control for such errors in our model. To do so, we will prioritize the maximization of **Recall** (TP/TP+FN) since it is an indicator of the dangerous False Negatives in our model but at the same time attempt to maximize **Precision** (TP/TP+FP) since it is an indicator of False Positives. 

### Logistic Regression

In [ ]:
lg = LogisticRegression()
lg.fit(X_train, y_train)

accuracy_lg = lg.score(X_test, y_test)*100
print('The accuracy for the Logistic Regression model is {}%'.format(accuracy_lg))

Let's attempt to improve the model using grid search.

In [ ]:
lg_parameters = {'tol':[.1, .001, .0001], 'C':[10, 1, .1]}
lg = LogisticRegression()
clf1 = GridSearchCV(lg, lg_parameters)

clf1.fit(X_train, y_train)
preds_lg = clf1.predict(X_test)
accuracy_lr = np.mean(y_test == preds_lg)*100


print('The accuracy for the refined Logistic Regression model is {}%'.format(accuracy_lr))

In [ ]:
# print(clf1.best_params_)
# print(clf1.best_score_)
# display(pd.DataFrame.from_dict(clf1.cv_results_).sort_values('rank_test_score'))

In [ ]:
target_names = ['Low Risk', 'Medium Risk', 'High Risk']
print(classification_report(y_test, preds_lg, target_names=target_names))

##### K-Fold Cross-Validation

In [ ]:
lr_cv = LogisticRegression()

cv_scores_LR = cross_val_score(lr_cv, X, y, cv=5)

print('Cross-Validation Scores: ' + str(cv_scores_LR))
print('Mean of Cross-Validation Scores: {}%'.format(np.mean(cv_scores_LR)*100))

##### Tuning Model Hyperparameters

In [ ]:
lr = LogisticRegression()

params = {'tol':[.1, .0001, 1e-5], 'C':[10, 1, .1]}

lr_gscv = GridSearchCV(lr, params, cv=5)

lr_gscv.fit(X_train, y_train)
preds_lr = lr_gscv.predict(X_test)
accuracy_lr_gscv = np.mean(y_test == preds_lr)*100

print('The accuracy for the refined Logistic Regression model is {}%'.format(accuracy_lr_gscv))

# NOTE: the following commented-out code is to print
# what the best parameter values are and 
# the mean cross-validated score of the best estimator
print('The best parameters are:')
for key, val in lr_gscv.best_params_.items():
    print(str(key) + ':', val)
# print('\nThe best score for the refined Logistic Regression model is {}%'.format(lr_gscv.best_score_*100))

##### Plotting ROC and P-R Curves

In [ ]:
# fit a model with the best parameters
lr_best = LogisticRegression(C=10, tol=0.0001)
lr_best.fit(X_train, y_train)

# plot multiclass P-R curve
visualize_data.plot_precision_recall(
    'Logistic Regression', lr_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

In [ ]:
# fit a model with the best parameters
lr_best = LogisticRegression(C=10, tol=0.0001)
lr_best.fit(X_train, y_train)

# plot multiclass ROC curve
visualize_data.plot_multiclass_roc(
    'Logistic Regression', lr_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

### K Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

accuracy_knn = knn.score(X_test, y_test)*100
print('The accuracy for the K Nearest Neighbors model is {}%'.format(accuracy_knn))

Let's attempt to improve the model using grid search.

In [ ]:
knn_parameters = {'n_neighbors':[10, 5, 3, 1], 'p':[2, 1]}
knn = KNeighborsClassifier()
clf2 = GridSearchCV(knn, knn_parameters)

clf2.fit(X_train, y_train)
preds_knn = clf2.predict(X_test)
accuracy_knn2 = np.mean(y_test == preds_knn)*100

print('The accuracy for the refined K Nearest Neighbors model is {}%'.format(accuracy_knn2))

In [ ]:
print(classification_report(y_test, preds_knn, target_names=target_names))

##### K-Fold Cross-Validation

In [ ]:
knn_cv = KNeighborsClassifier()

cv_scores = cross_val_score(knn_cv, X, y, cv=5)

print('Cross-Validation Scores: ' + str(cv_scores))
print('Mean of Cross-Validation Scores: {}%'.format(np.mean(cv_scores)*100))

##### Tuning Model Hyperparameters

In [ ]:
knn = KNeighborsClassifier()

params = {'n_neighbors': [10, 5, 3, 1], 'p':[3, 2, 1]}

knn_gscv = GridSearchCV(knn, params, cv=5)

knn_gscv.fit(X_train, y_train)
preds_knn = knn_gscv.predict(X_test)
accuracy_knn_gscv = np.mean(y_test == preds_knn)*100

print('The accuracy for the refined K Nearest Neighbors model is {}%'.format(accuracy_knn_gscv))

# NOTE: the following commented-out code is to print
# what the best parameter values are and 
# the mean cross-validated score of the best estimator
print('The best parameters are:')
for key, val in knn_gscv.best_params_.items():
    print(str(key) + ':', val)
# print('\nThe best score for the refined K Nearest Neighbors model is {}%'.format(knn_gscv.best_score_*100))

##### Plotting ROC and P-R Curves

In [ ]:
# fit a model with the best parameters
knn_best = KNeighborsClassifier(n_neighbors=3, p=3)
knn_best.fit(X_train, y_train)

# plot multiclass P-R curve
visualize_data.plot_precision_recall(
    'K Nearest Neighbors', knn_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

In [ ]:
# fit a model with the best parameters
knn_best = KNeighborsClassifier(n_neighbors=3, p=3)
knn_best.fit(X_train, y_train)

# plot multiclass ROC curve
visualize_data.plot_multiclass_roc(
    'K Nearest Neighbors', knn_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

### Support Vector Machine

In [ ]:
svc = SVC()
svc.fit(X_train, y_train)

accuracy_svc = svc.score(X_test, y_test)*100
print('The accuracy for the Support Vector Machine model is {}%'.format(accuracy_svc))

In [ ]:
svc_parameters = {'tol': [.1, .001, .0001], 'C':[10, 1, .1]}
svc = SVC()
clf3 = GridSearchCV(svc, svc_parameters)

clf3.fit(X_train, y_train)
preds_svc = clf3.predict(X_test)
accuracy_svc2 = np.mean(y_test == preds_svc)*100

print('The accuracy for the refined Support Vector Machine model is {}%'.format(accuracy_svc2))

In [ ]:
print(classification_report(y_test, preds_svc, target_names=target_names))

##### K-Fold Cross-Validation

In [ ]:
svc_cv = SVC()

cv_scores = cross_val_score(svc_cv, X, y, cv=5)

print('Cross-Validation Scores: ' + str(cv_scores))
print('Mean of Cross-Validation Scores: {}%'.format(np.mean(cv_scores)*100))

##### Tuning Model Hyperparameters

In [ ]:
svc = SVC()
params = {'tol': [.1, .001, .0001], 'C':[10, 1, .1]}
svc_gscv = GridSearchCV(svc, params, cv=5)

svc_gscv.fit(X_train, y_train)
preds_svc = svc_gscv.predict(X_test)
accuracy_svc_gscv = np.mean(y_test == preds_svc)*100

print('The accuracy for the refined Support Vector Machine model is {}%'.format(accuracy_svc_gscv))

# NOTE: the following commented-out code is to print
# what the best parameter values are and 
# the mean cross-validated score of the best estimator
print('The best parameters are:')
for key, val in svc_gscv.best_params_.items():
    print(str(key) + ':', val)
# print('\nThe best score for the refined Support Vector Machine model is {}%'.format(svc_gscv.best_score_*100))

##### Plotting ROC and P-R Curves

In [ ]:
# fit a model with the best parameters
svc_best = SVC(C=10, tol=0.1)
svc_best.fit(X_train, y_train)

# plot multiclass P-R curve
visualize_data.plot_precision_recall(
    'Support Vector Machine', svc_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

In [ ]:
# fit a model with the best parameters
svc_best = SVC(C=10, tol=0.1)
svc_best.fit(X_train, y_train)

# plot multiclass ROC curve
visualize_data.plot_multiclass_roc(
    'Support Vector Machine', svc_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

### Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

accuracy_gnb = gnb.score(X_test, y_test)*100
print('The accuracy for the Naive Bayes is {}%'.format(accuracy_gnb))

In [ ]:
gnb.get_params()

In [ ]:
gnb_parameters = {'var_smoothing':[1e-3, 1e-6, 1e-9]}
gnb = GaussianNB()
clf4 = GridSearchCV(gnb, gnb_parameters)

clf4.fit(X_train, y_train)
preds_gnb = clf4.predict(X_test)
accuracy_gnb2 = np.mean(y_test == preds_gnb)*100

print('The accuracy for the refined Naive Bayes model is {}%'.format(accuracy_gnb2))

In [ ]:
print(classification_report(y_test, preds_gnb, target_names=target_names))

##### K-Fold Cross-Validation

In [ ]:
# perform k-fold cross-validation
gnb_cv = GaussianNB()

%time cv_scores = cross_val_score(gnb_cv, X, y, cv=5)

print('Cross-Validation Scores: ' + str(cv_scores))
print('Mean of Cross-Validation Scores: {}%'.format(np.mean(cv_scores)*100))

##### Tuning Model Hyperparameters

In [ ]:
# perform hyperparameter tuning and output the best params and score
gnb = GaussianNB()

# the 'priors' values come from the following:
# - the first list of values is just indicating 
# equal weights between the 3 classes, which are 0, 1, and 2
# - the second list of value refers to the weights
# of the classes that we provided initially
params = {#'priors': [[0.333, 0.333, 0.334], [0.55, 0.30, 0.15]],
          'var_smoothing': [0,1e-3,1e-6, 1e-9,0.01,0.1,0.5,1]}

gnb_gscv = GridSearchCV(gnb, params, cv=5)

gnb_gscv.fit(X_train, y_train)

preds_gnb = gnb_gscv.predict(X_test)
accuracy_gnb_gscv = np.mean(y_test == preds_gnb)*100

print('The accuracy for the refined Naive Bayes model is {}%'.format(accuracy_gnb_gscv))

# NOTE: the following commented-out code is to print
# what the best parameter values are and 
# the mean cross-validated score of the best estimator
print('The best parameters are:')
for key, val in gnb_gscv.best_params_.items():
    print(str(key) + ':', val)
# print('\nThe best score for the refined Naive Bayes model is {}%'.format(gnb_gscv.best_score_*100))

##### Plotting ROC and P-R Curves

In [ ]:
# fit a model with the best parameters
gnb_best = GaussianNB(priors=[0.333, 0.333, 0.334], var_smoothing=0.1)
gnb_best.fit(X_train, y_train)

# plot multiclass P-R curve
visualize_data.plot_precision_recall(
    'Naive Bayes', gnb_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

In [ ]:
# fit a model with the best parameters
gnb_best = GaussianNB(priors=[0.333, 0.333, 0.334], var_smoothing=0.1)
gnb_best.fit(X_train, y_train)

# plot multiclass ROC curve
visualize_data.plot_multiclass_roc(
    'Naive Bayes', gnb_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

### Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

accuracy_rf = rf.score(X_test, y_test)*100
print('The accuracy for the Random Forest is {}%'.format(accuracy_rf))

In [ ]:
rf_parameters = {'n_estimators':[100, 50, 10]}
rf = RandomForestClassifier()
clf5 = GridSearchCV(rf, rf_parameters)

clf5.fit(X_train, y_train)
preds_rf = clf5.predict(X_test)
accuracy_rf2 = np.mean(y_test == preds_rf)*100

print('The accuracy for the refined Random Forest model is {}%'.format(accuracy_rf2))

In [ ]:
print(classification_report(y_test, preds_rf, target_names=target_names))

##### K-Fold Cross-Validation

In [ ]:
# perform k-fold cross-validation
rf_cv = RandomForestClassifier()

%time cv_scores = cross_val_score(rf_cv, X, y, cv=5)

print('Cross-Validation Scores: ' + str(cv_scores))
print('Mean of Cross-Validation Scores: {}%'.format(np.mean(cv_scores)*100))

##### Tuning Model Hyperparameters

In [ ]:
# perform hyperparameter tuning and output the best params and score
rf = RandomForestClassifier()

# the 'priors' values come from the following:
# - the first list of values is the default, which just indicates 
# equal weights between the 3 classes, which are 0, 1, and 2
# - the second list of value refers to the weights
# of the classes that we provided initially
params = {#'class_weight': [{0: 1, 1: 1, 2: 1}, {0: 0.55, 1: 0.30, 2: 0.15}],
          'n_estimators':[200, 100, 50, 10]}

rf_gscv = GridSearchCV(rf, params, cv=5)

rf_gscv.fit(X_train, y_train)

preds_rf = rf_gscv.predict(X_test)
accuracy_rf_gscv = np.mean(y_test == preds_rf)*100

print('The accuracy for the refined Random Forest model is {}%'.format(accuracy_rf_gscv))

# NOTE: the following commented-out code is to print
# what the best parameter values are and 
# the mean cross-validated score of the best estimator
print('The best parameters are:')
for key, val in rf_gscv.best_params_.items():
    print(str(key) + ':', val)
# print('\nThe best score for the refined Random Forest model is {}%'.format(rf_gscv.best_score_*100))

##### Plotting ROC and P-R Curves

In [ ]:
# fit a model with the best parameters
rf_best = RandomForestClassifier(class_weight={0: 0.55, 1: 0.3, 2: 0.15}, 
                                 n_estimators=200)
rf_best.fit(X_train, y_train)

# plot multiclass P-R curve
visualize_data.plot_precision_recall(
    'Random Forest', rf_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

In [ ]:
# fit a model with the best parameters
rf_best = RandomForestClassifier(class_weight={0: 0.55, 1: 0.3, 2: 0.15}, 
                                 n_estimators=200)
rf_best.fit(X_train, y_train)

# plot multiclass ROC curve
visualize_data.plot_multiclass_roc(
    'Random Forest', rf_best, X_test, y_test, 
    n_classes=3, figsize=(16, 10))

### Model Comparison

In [ ]:
models = ['Logistic Regression', 'K Nearest Neighbors',
         'Support Vector Machine', 'Naive Bayes', 'Random Forest']
model_accuracies = [accuracy_lg, accuracy_knn, 
                    accuracy_svc, accuracy_gnb, accuracy_rf]
gscv_accuracies = [accuracy_lr_gscv, accuracy_knn_gscv,
                  accuracy_svc_gscv, accuracy_gnb_gscv, accuracy_rf_gscv]
model_comparison = pd.DataFrame(
    np.column_stack([models, model_accuracies, gscv_accuracies]),
    columns=['Model', 'Model Accuracy', 'Grid Search CV Accuracy'])
model_comparison.head()